<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/confrontoFumatoriTitolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confronto fumatori per Titolo di studio

## Grafico a barre orizzontali

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os

# Percorso
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetFumatoriTitolo(istogramma).csv")

try:
    # Caricamento
    df = pd.read_csv(
        file_path,
        sep=';',
        encoding='latin-1',
        quoting=csv.QUOTE_NONE,
        on_bad_lines='skip'
    )

    # Pulizia
    df.columns = [c.strip().replace('"', '').replace("'", "") for c in df.columns]

    # Indici colonne 11 - Titolo di studio, 13 - Valore
    col_titolo_idx = 11
    col_valore_idx = 13

    # Pulizia
    data = pd.DataFrame({
        'Titolo_Studio': df.iloc[:, col_titolo_idx].astype(str).str.replace('"', '').str.strip(),
        'Fumatori': df.iloc[:, col_valore_idx]
    })

    # Conversione numerica
    data['Fumatori'] = pd.to_numeric(data['Fumatori'], errors='coerce')
    data = data.dropna()

    # Conversione in milioni
    data['Fumatori_M'] = data['Fumatori'] / 1000

    # Invertiamo l'ordine per avere la "gerarchia scolastica" al contrario
    data = data.iloc[::-1]

    # Configurazione grafica
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(12, 7), dpi=100)

    # Colori
    sky_blue = '#5DADE2'   # Blu
    text_color = '#2E4053' # Grigio scuro
    sub_color = '#95A5A6'  # Grigio per gli assi

    # Grafico a barre orizzontali
    bars = ax.barh(
        data['Titolo_Studio'],
        data['Fumatori_M'],
        color=sky_blue,
        height=0.7,    # Barre più strettine
        edgecolor='none',
        zorder=3
    )

    # Valori a lato di ogni barra
    for bar in bars:
        width = bar.get_width()
        ax.annotate(
            f'{width:.2f}M',
            xy=(width, bar.get_y() + bar.get_height() / 2),
            xytext=(10, 0),
            textcoords="offset points",
            ha='left', va='center',
            fontsize=11,
            fontweight='bold',
            color=text_color
        )

    # Titolo
    plt.title('Distribuzione Fumatori per Titolo di Studio',
              fontsize=20, pad=40, fontweight='bold', color=text_color, loc='left')

    # Sottotitolo
    ax.text(0, 1.06, 'Dati del 2024 • Valori espressi in milioni',
            transform=ax.transAxes,
            fontsize=11,
            color=sub_color,
            fontweight='normal')

    # Pulizia degli assi
    ax.set_xlabel('Fumatori (milioni)', fontsize=10, color=sub_color, labelpad=15)
    ax.set_ylabel('') # I nomi sono abbastanza autoesplicativi

    # Rimuove i bordi
    sns.despine(left=True, bottom=True)

    # Griglia solo per l'asse x
    ax.xaxis.grid(True, linestyle='-', alpha=0.1, color='black', zorder=0)
    ax.yaxis.grid(False)

    # Font e etichette
    plt.yticks(fontsize=11, color=sub_color)
    # Formatter per l'asse x
    def x_axis_formatter(x, pos):
        return f'{x:.0f}M'
    ax.xaxis.set_major_formatter(plt.FuncFormatter(x_axis_formatter))
    plt.xticks(fontsize=10, color=sub_color)

    # Aggiunge margine a destra per le etichette
    ax.set_xlim(0, data['Fumatori_M'].max() * 1.15)

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Errore durante l'elaborazione: {e}")

## Grafico a linee

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os

# Percorso
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetFumatoriTitolo(linee).csv")


try:
    # Caricamento
    df = pd.read_csv(
        file_path,
        sep=';',
        encoding='latin-1',
        quoting=csv.QUOTE_NONE,
        on_bad_lines='skip'
    )

    # Pulizia nomi colonne
    df.columns = [c.strip().replace('"', '').replace("'", "") for c in df.columns]

    # Mappatura indici
    col_id_idx = 10
    col_label_idx = 11
    col_anno_idx = 12
    col_valore_idx = 13

    # Estrazione dati
    raw_data = pd.DataFrame({
        'ID': df.iloc[:, col_id_idx].astype(str).str.strip(),
        'Label': df.iloc[:, col_label_idx].astype(str).str.strip(),
        'Anno': pd.to_numeric(df.iloc[:, col_anno_idx], errors='coerce'),
        'Valore': pd.to_numeric(df.iloc[:, col_valore_idx], errors='coerce')
    }).dropna()

    # Raggruppamento
    def map_education(row):
        rid = row['ID']
        label = row['Label'].lower()
        if 'laurea' in label or rid == '11':
            return 'Laurea e post-laurea'
        if 'diploma' in label or rid == '5':
            return 'Diploma superiore'
        if 'media' in label or rid == '4':
            return 'Licenza media'
        if 'elementare' in label or 'nessun' in label or rid == '3':
            return 'Licenza elementare / Nessuno'
        return None

    raw_data['Categoria'] = raw_data.apply(map_education, axis=1)
    filtered_data = raw_data[raw_data['Categoria'].notnull()].copy()

    # Ordinamento
    final_data = filtered_data.groupby(['Anno', 'Categoria'])['Valore'].sum().reset_index()
    final_data = final_data.sort_values(by=['Anno'])
    final_data['Valore_M'] = final_data['Valore'] / 1000

    ## --- LOGICA AFFINATA PER EVITARE SOVRAPPOSIZIONI DEI NUMERI SOPRA I PALLINI ---
    ## Creiamo una chiave di controllo arrotondata alla prima cifra decimale (X.Y)
    #final_data['Check_Sovrapposizione'] = final_data['Valore_M'].round(1)

    ## Per ogni anno e ogni valore X.Y, identifichiamo l'indice del record che ha il valore ORIGINALE (dal CSV) più alto.
    #indici_da_annotare = final_data.groupby(['Anno', 'Check_Sovrapposizione'])['Valore'].idxmax().values
    ## --------------------- FINE -----------------------------

    # Configurazione Grafica
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(16, 10), dpi=100)

    colors = {
        'Laurea e post-laurea': '#27AE60',
        'Diploma superiore': '#2980B9',
        'Licenza media': '#F39C12',
        'Licenza elementare / Nessuno': '#C0392B'
    }

    categories = ['Laurea e post-laurea', 'Diploma superiore', 'Licenza media', 'Licenza elementare / Nessuno']

    for cat in categories:
        cat_df = final_data[final_data['Categoria'] == cat]
        if not cat_df.empty:
            plt.plot(
                cat_df['Anno'],
                cat_df['Valore_M'],
                label=cat,
                color=colors[cat],
                linewidth=2.5,
                marker='o',
                markersize=6,
                markeredgecolor='white',
                zorder=3
            )

            #-------------Iterazione 2.0-----------------------
            # AGGIUNTA VALORI SU OGNI PUNTO (FILTRATI PER PRECISIONE X.Y)
            #for row in cat_df.itertuples():
            #    # Verifichiamo se questo punto è quello col valore CSV più alto nel suo "gruppo di vicinanza"
            #    if row.Index in indici_da_annotare:
            #        ax.annotate(
            #            f'{row.Valore_M:.1f}M',
            #            (row.Anno, row.Valore_M),
            #            textcoords="offset points",
            #            xytext=(0, 8),
            #            ha='center',
            #            fontsize=8,
            #            fontweight='bold',
            #            color=colors[cat],
            #            alpha=0.9
            #        )
            #----------------------------------------------------

            # Valori su ogni pallino
            for x, y in zip(cat_df['Anno'], cat_df['Valore_M']):
                ax.annotate(
                    f'{y:.1f}M',
                    (x, y),
                    textcoords="offset points",
                    xytext=(0, 8), # Spostamento verso l'alto rispetto al pallino
                    ha='center',
                    fontsize=8,
                    fontweight='bold',
                    color=colors[cat],
                    alpha=0.9
                )

    # Titolo
    plt.title('Andamento Fumatori in Italia per Titolo di Studio',
              fontsize=22, pad=60, fontweight='bold', color='#2E4053', loc='left')

    # Sottotitolo
    ax.text(0, 1.08, 'Dati dal 2001 al 2024 • Valori espressi in milioni',
            transform=ax.transAxes, fontsize=12, color='#7F8C8D')

    # Assi
    plt.xlabel('Anno di rilevazione', fontsize=11, labelpad=15, color='#95A5A6')
    plt.ylabel('Fumatori (milioni)', fontsize=11, labelpad=15, color='#95A5A6')

    plt.legend(frameon=False, fontsize=10, loc='upper left', bbox_to_anchor=(0, 1.06), ncol=4)

    sns.despine(left=True, bottom=True)
    ax.yaxis.grid(True, linestyle='--', alpha=0.2, color='#BDC3C7')

    years = sorted(final_data['Anno'].unique().astype(int))
    plt.xticks(years, rotation=45, fontsize=9, color='#7F8C8D')
    plt.yticks(fontsize=10, color='#7F8C8D')

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Errore: {e}")